# ChannelMutator
A channel mutator is a manager of the channel structure of a model. In other words, it manages all MutableChannelUnits of a model.  
ChannelMutator is the simplest channel mutator. All other channel mutators should inherit from ChannelMutator class. We take ChannelMutator as an example.

##  How to Construct a ChannelMutator

Suppose we have a model archtecture defineed below

In [24]:
# define a model
from mmengine.model import BaseModel
from torch import nn
import torch
from collections import OrderedDict

class MyModel(BaseModel):

    def __init__(self):
        super().__init__(None, None)
        self.net = nn.Sequential(
            OrderedDict([('conv0', nn.Conv2d(3, 8, 3, 1, 1)),
                         ('relu', nn.ReLU()),
                         ('conv1', nn.Conv2d(8, 16, 3, 1, 1))]))
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Linear(16, 1000)

    def forward(self, x):
        feature = self.net(x)
        pool = self.pool(feature).flatten(1)
        return self.head(pool)

There are two steps to fully constructing a ChannelMutator object as below. 
1. we need to initialize a ChannelMutator object.
2. Then we need to init the ChannelMutator object with a model.

In [25]:
from mmrazor.models.mutators import ChannelMutator

model = MyModel()
# initialize a ChannelMutator object
mutator = ChannelMutator(
    channel_unit_cfg=dict(
        type='SequentialMutableChannelUnit',
        default_args=dict(choice_mode='ratio'),
        units={},
    ),
    parse_cfg=dict(
        type='BackwardTracer',
        loss_calculator=dict(type='ImageClassifierPseudoLoss')))
# init the ChannelMutator object with a model
mutator.prepare_from_supernet(model)
print(f'The mutator has {len(mutator.mutable_units)} mutable channel units.')

The mutator has 2 mutable channel units.


ChannelMutator has two arguments:
1. channel_unit_cfg: config of the MutableChannelUnit to use in the ChannelMutator.
2. parse_cfg: the way to parse the model and get MutableChannelUnits.

There are there ways to parse model and get MutableChannelUnits.
1. Use a tracer to get MutableChannelUnits automatically.
2. Use config dicts to indicate MutableChannelUnits.
3. Predefine MutableChannels in the model archtecture.
   
The example of method 1 has been post above. We post the examples of method 2 and method 3 below.

In [26]:
# 2. use config dicts to indicate MutableChannelUnits.
from mmrazor.models.mutators import ChannelMutator

model = MyModel()
# initialize a ChannelMutator object
mutator = ChannelMutator(
    channel_unit_cfg=dict(
        type='SequentialMutableChannelUnit',
        default_args=dict(choice_mode='ratio'),
        units={
            'net.conv0_(0, 8)_8': {
                'init_args': {
                    'num_channels': 8,
                },
                'channels': {
                    'input_related': [{
                        'name': 'net.conv1',
                    }],
                    'output_related': [{
                        'name': 'net.conv0',
                    }]
                },
                'choice': 1.0
            },
            'net.conv1_(0, 16)_16': {
                'init_args': {
                    'num_channels': 16,
                },
                'channels': {
                    'input_related': [{
                        'name': 'head',
                    }],
                    'output_related': [{
                        'name': 'net.conv1',
                    }]
                },
                'choice': 1.0
            }
        }),
    parse_cfg=dict(type='Config'))
# init the ChannelMutator object with a model
mutator.prepare_from_supernet(model)
print(f'The mutator has {len(mutator.mutable_units)} mutable channel units.')

The mutator has 2 mutable channel units.


In [27]:
# 3. Predefine MutableChannels in the model archtecture.

from mmrazor.models.architectures.dynamic_ops import DynamicConv2d, DynamicLinear
from mmrazor.models.mutables import MutableChannelUnit, MutableChannelContainer, SquentialMutableChannel
from collections import OrderedDict

class MyDynamicModel(BaseModel):

    def __init__(self):
        super().__init__(None, None)
        self.net = nn.Sequential(
            OrderedDict([('conv0', DynamicConv2d(3, 8, 3, 1, 1)),
                         ('relu', nn.ReLU()),
                         ('conv1', DynamicConv2d(8, 16, 3, 1, 1))]))
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.head = DynamicLinear(16, 1000)

        # register MutableChannelContainer
        MutableChannelUnit._register_channel_container(
            self, MutableChannelContainer)
        self._register_mutables()

    def forward(self, x):
        feature = self.net(x)
        pool = self.pool(feature).flatten(1)
        return self.head(pool)

    def _register_mutables(self):
        mutable1 = SquentialMutableChannel(8)
        mutable2 = SquentialMutableChannel(16)
        MutableChannelContainer.register_mutable_channel_to_module(
            self.net.conv0, mutable1, is_to_output_channel=True)
        MutableChannelContainer.register_mutable_channel_to_module(
            self.net.conv1, mutable1, is_to_output_channel=False)

        MutableChannelContainer.register_mutable_channel_to_module(
            self.net.conv1, mutable2, is_to_output_channel=True)
        MutableChannelContainer.register_mutable_channel_to_module(
            self.head, mutable2, is_to_output_channel=False)


model = MyDynamicModel()
# initialize a ChannelMutator object
mutator = ChannelMutator(
    channel_unit_cfg=dict(
        type='SequentialMutableChannelUnit',
        default_args=dict(choice_mode='ratio'),
        units={},
    ),
    parse_cfg=dict(type='Predefined'))
# init the ChannelMutator object with a model
mutator.prepare_from_supernet(model)
print(f'The mutator has {len(mutator.mutable_units)} mutable channel units.')

The mutator has 2 mutable channel units.


## How to Change the Structure of a Model

The structure of a model is represented by a dict where the key is the name of a MutableChannelUnit and the value is a structure choice.

In [28]:
print(mutator.current_choices)

{0: 8, 1: 16}


We can change the dict to prune the model.

In [29]:
mutator.set_choices(
    {0: 4, 1: 8}
)
print(model)

MyDynamicModel(
  (data_preprocessor): BaseDataPreprocessor()
  (net): Sequential(
    (conv0): DynamicConv2d(
      3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (mutable_attrs): ModuleDict(
        (in_channels): MutableChannelContainer(num_channels=3, activated_channels=3)
        (out_channels): MutableChannelContainer(num_channels=8, activated_channels=4)
      )
    )
    (relu): ReLU()
    (conv1): DynamicConv2d(
      8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (mutable_attrs): ModuleDict(
        (in_channels): MutableChannelContainer(num_channels=8, activated_channels=4)
        (out_channels): MutableChannelContainer(num_channels=16, activated_channels=8)
      )
    )
  )
  (pool): AdaptiveAvgPool2d(output_size=1)
  (head): DynamicLinear(
    in_features=16, out_features=1000, bias=True
    (mutable_attrs): ModuleDict(
      (in_features): MutableChannelContainer(num_channels=16, activated_channels=8)
      (out_features): MutableChannelCo

Please refer to our documents for more choices related methods.